# ElasticSearch

In this notebook we have created a search engine using ElasticSearch. 

For our own reference:
* Literature: <https://www.elastic.co/guide/en/elasticsearch/guide/current/index.html>
* Telegraaf XML documents: http://data.politicalmashup.nl/arjan/telegraaf/



Create a search engine for the telegraaf newspaper collection using eg ElasticSearch. Make facets for years and document types. Pay attention to telephone numbers (in mini advertisements). Hieronder een voorbeeld van 1 document (= 1 artikeltje).
Je ziet dat er zelfs een link naar de bron tekst (als plaatje) instaat. De URL linked door naar de nieuwe url http://www.delpher.nl/nl/kranten/view?identifier=ddd%3A010563762%3Ampeg21%3Aa0005&coll=ddd ElasticSearch gebruikt een JSON formaat als invoer, en dit is dus triviaal om te zetten naar JSON.

Each of the following points must be addressed. Create a seperate page on the wiki for each point. Make sure these pages can be found from the menu of your wiki. Explain what you did, and exemplify with links to screenshots/a working system.

* Search as we know it from Google. Give a result page (SERP), with links to the documents and some description of each hit.
* Advanced search. Let a user be able to search in several fields, also in several fields simulteanously. Queries like "return kamervragen by Wilders about XXX with an answer about YYY in the period ZZZ" should be possible. (For the "Telegraaf" collectie, let the user search in both title and tekst fields)
* Do one of the following:
    1. Represent the hits of a query with a wordcloud of 25-50 informative words. The wordcloud should somehow summarise what the collection has to say about the query. You may think of these words as words that you could add to the query in order to improve recall (blind relevance feedback/query expansion).
    2. Represent each document (a kamervraag) with a word-cloud. Also make word-clouds for the question and for the answer. EXAMPLE: The html files in http://data.politicalmashup.nl/arjan/odeii/data_as_html/ contain such wordcloud summaries, which work rather well.   

You can use several techniques to get rid of high frequency, but meaningless words: of course IDF, but also mutual information (see 13.5.1), or of course the technique from the paper by Kaptein et al on wordclouds.

* Give next to a traditional list of results, a timeline in which you indicate how many hits there are over time.
* Give next to the traditional list of results, a table with the number of hits for each political party. Link the party names, which should result in only selecting the hits "ingediend" by members of that party. (Faceted Search) (For the "Telegraaf" collectie, use the dc:subject element as facet values.)
* Evaluate your results Let 2 persons assess the relevancy of the top 10 documents for 5 different queries. Compute Cohen's kappa. Determine the average precision at 10 for your system based on these 10 queries, and the two relevance assesments. Also plot the P@10 (for both judges) for each query, showing differences in hard and easy queries. Describe clearly how you solved differences in judgements. 
Create your queries in the following format:

                    <topic number="6"  >
          <query>kcs</query>
          <description>Find information on the Kansas City Southern railroad.
          </description>
           
        </topic>

        <topic number="16"  >
          <query>arizona game and fish</query>
          <description>I'm looking for information about fishing and hunting
          in Arizona.
          </description>
           
        </topic>
                

So, both provide the actual query, and a description of the information need that was behind the query.
Give a small set of clear guidelines for judging the results, and let your judges follow these guidelines.
It is far more interesting to have difficult queries (both for the search engine and for the judges) than to have queries on which all ten retrieved documents are relevant. So, try to create a good list of information needs.

* Change the ranking of your system, compute the average precision at 10 using your 10 queries, compare the results to your old system, and EXPLAIN what is going on.


# The Search Engine

Before running ES run: 

    export ES_HEAP_SIZE=Half_RAM

where Half_RAM is half your ram

AND: 
in /config/elasticsaerch.yml add 
indices.memory.index_buffer_size: 50% 
(Still need to check if this makes a difference)

To start the Elastic searh serive, please run the following code in commandline:

    ./elasticsearch-2.4.1/bin/elasticsearch --node.name telegraaf

## Initiate connection to the Elastic Search engine

In [36]:
import sys
import json
from elasticsearch import Elasticsearch, helpers

HOST = 'http://localhost:9200/'
es = Elasticsearch(hosts=[HOST],retry_on_timeout=True)

# If code runs, the connection is made

## Generator to read Telegraaf XML and add them to the ES database

A generator makes it possible to immediately put the XML files/documents into the ES databse

* Remove high frequency, but meaningless words

You can use several techniques to get rid of high frequency, but meaningless words: of course IDF, but also mutual information (see 13.5.1), or of course the technique from the paper by Kaptein et al on wordclouds.

In [2]:
from bs4 import BeautifulSoup
import sys
from os import listdir
from os.path import isfile
import xml.etree.ElementTree as etree
from lxml import etree


def read(year):
    '''
    return a generator for the date, subject(type), 
    title, and text for each item in the given year. 
    '''
    count = 0
    date = ''
    subject =''
    identifier = ''
    text = ''
    title = ''
    for event, elem in etree.iterparse(year,events=("start", "end")):
        if event =='end':
            if elem.tag == '{http://purl.org/dc/elements/1.1/}date':
                date = elem.text
                elem.clear()
            elif elem.tag == '{http://purl.org/dc/elements/1.1/}subject':
                subject = elem.text
                elem.clear()
            elif elem.tag == '{http://purl.org/dc/elements/1.1/}identifier':
                identifier = elem.text
                elem.clear()
            elif elem.tag == 'title':
                if elem.text == None:
                    title = ''
                else:
                    title = elem.text
                elem.clear()
            elif elem.tag == 'p':
                if elem.text == None:
                    text = ''
                else:
                    text = elem.text
                elem.clear()
            elif elem.tag == '{http://www.politicalmashup.nl}root':
                elem.clear()
                yield (date,subject,identifier,title,text)
            else:
                elem.clear()
                
#     soup = BeautifulSoup(open(year,'r'),'xml')
#     for date,subject, title, text, identifier in zip(soup.find_all('date'), soup.find_all('subject'), 
#                                                      soup.find_all('title'), soup.find_all('text'),
#                                                      soup.find_all('identifier')):
#             yield (date.text,subject.text,title.text,text.text,identifier.text)

documents = ['./Telegraaf/'+i for i in listdir('./Telegraaf') if not isfile(i)]

print(documents)

# Create the generator for the bulk importer
# I'm not sure if it's a good idea to use _type here as a subject (which is artcle or advertisement, or more...)
# The score calculation for the Elastic Search database uses whole-index statistics. 
# If you're searching a subsection this will alter the scores! WE WILL NEED TO KEEP THIS IN MIND.
#k = ({'_type':subject, '_index':'telegraaf','_source':{'year':date[:4], 'date':date[5:], 'title':title, 'text':text}} 
#    for year in documents for (date,subject,title,text) in read(year))


['./Telegraaf/telegraaf-1918.xml', './Telegraaf/telegraaf-1922.xml', './Telegraaf/telegraaf-1923.xml', './Telegraaf/telegraaf-1951.xml', './Telegraaf/telegraaf-1961.xml', './Telegraaf/telegraaf-1962.xml', './Telegraaf/telegraaf-1963.xml', './Telegraaf/telegraaf-1965.xml', './Telegraaf/telegraaf-1966.xml', './Telegraaf/telegraaf-1968.xml', './Telegraaf/telegraaf-1969.xml', './Telegraaf/telegraaf-1970.xml', './Telegraaf/telegraaf-1971.xml', './Telegraaf/telegraaf-1972.xml', './Telegraaf/telegraaf-1973.xml', './Telegraaf/telegraaf-1974.xml', './Telegraaf/telegraaf-1975.xml', './Telegraaf/telegraaf-1976.xml', './Telegraaf/telegraaf-1977.xml', './Telegraaf/telegraaf-1978.xml', './Telegraaf/telegraaf-1979.xml', './Telegraaf/telegraaf-1980.xml', './Telegraaf/telegraaf-1981.xml', './Telegraaf/telegraaf-1982.xml', './Telegraaf/telegraaf-1983.xml', './Telegraaf/telegraaf-1984.xml', './Telegraaf/telegraaf-1985.xml', './Telegraaf/telegraaf-1986.xml', './Telegraaf/telegraaf-1987.xml', './Telegraaf/

## Populate ES database

In [3]:
# List of all indices
! curl 'localhost:9200/_cat/indices?v'

health status index                   pri rep docs.count docs.deleted store.size pri.store.size 
yellow open   .marvel-es-1-2016.10.26   1   1       7007           88      4.4mb          4.4mb 
yellow open   .marvel-es-1-2016.10.25   1   1       6010          100      2.1mb          2.1mb 
yellow open   .marvel-es-1-2016.10.24   1   1      45705           72     16.6mb         16.6mb 
yellow open   .marvel-es-1-2016.10.23   1   1       4619           96      1.9mb          1.9mb 
yellow open   .kibana                   1   1          2            0       19kb           19kb 
yellow open   .marvel-es-data-1         1   1         29            6     26.4kb         26.4kb 
yellow open   telegraaf                 5   1    5655966            0     10.9gb         10.9gb 


In [4]:
# delete any pre-excisting index
#es.indices.delete(index='telegraaf', ignore=[404,400])

{u'acknowledged': True}

In [5]:
# Create the telegraaf index in our telegraaf node
es.indices.create(index='telegraaf', ignore=400)

{u'acknowledged': True}

In [6]:
# turn refresh to a less frequent rate to speed up bulk import
es.indices.put_settings(index='telegraaf',body={"index" : 
                                            {"refresh_interval" : "30s"
                                            }
                                       })

{u'acknowledged': True}

In [7]:
import time

#Import the information into the database
# bundeling same type together could speed up bulk process
# I
def bulk_per_doc(documents):
    start = time.time()
    print "Starting time:", start
    for doc in documents:
        print "Document:", doc
        failed = 0
        succes = 0
        start_doc = time.time()
        k = ({'_index':'telegraaf', '_type':subject, '_id':identifier, '_source':{'year':date[:4], 
               'date':date[5:], 'title':title, 'text':text}} for (date,subject,identifier,title,text) in read(doc))
        for (ok, res) in helpers.parallel_bulk(es,k, chunk_size=4000,max_chunk_bytes=15728640, request_timeout=10000):
            if not ok:
                failed +=1
            else:
                succes += 1
        end_doc = time.time()
        print '[Done]', end_doc-start_doc
        print "Succesful injections: ", succes
        print "Failed injections:", failed
        print '\n'
    end =time.time()
    print "Finished", (end_doc-start)

bulk_per_doc(documents)    

# It is better to use the per doc populate method, because otherwise the bulk queue will overflow. 
# def bulk_per_doc(documents):
#     start = time.time()
#     print "Starting time:", start

#     for i in documents[:1]:
#         start_doc = time.time()
#         k = ({'_type':subject, '_id':identifier, '_index':'telegraaf','_source':{'year':date[:4], 
#              'date':date[5:], 'title':title, 'text':text}}
#              for (date,subject,title,text,identifier) in read(i))
#         for ok in helpers.parallel_bulk(es,k,chunk_size=500,max_chunk_bytes=15728640):
#             continue
#         end_doc =time.time()
#         print "Finished", (end_doc-start_doc)
#         end = time.time()
#     print "Done:", end - start
        


Starting time: 1477420650.39
Document: ./Telegraaf/telegraaf-1971.xml
[Done] 97.113476038
Succesful injections:  174873
Failed injections: 0


Document: ./Telegraaf/telegraaf-1979.xml
[Done] 97.8079650402
Succesful injections:  149075
Failed injections: 0


Document: ./Telegraaf/telegraaf-1974.xml
[Done] 93.9970080853
Succesful injections:  137155
Failed injections: 0


Document: ./Telegraaf/telegraaf-1994.xml
[Done] 150.787832022
Succesful injections:  251526
Failed injections: 0


Document: ./Telegraaf/telegraaf-1990.xml
[Done] 138.377987146
Succesful injections:  228202
Failed injections: 0


Document: ./Telegraaf/telegraaf-1981.xml
[Done] 124.811862946
Succesful injections:  203119
Failed injections: 0


Document: ./Telegraaf/telegraaf-1963.xml
[Done] 81.1194469929
Succesful injections:  120901
Failed injections: 0


Document: ./Telegraaf/telegraaf-1976.xml
[Done] 129.643987179
Succesful injections:  179814
Failed injections: 0


Document: ./Telegraaf/telegraaf-1989.xml
[Done] 157.

In [93]:
# Set the refresh rate back to default
es.indices.put_settings(index='telegraaf',body={"index" : 
                                            {"refresh_interval" : "1s"
                                            }
                                       })

{u'acknowledged': True}

In [6]:
# Speed improvements/performance: 
# - bootstrap.mlockall: true in config of the file 
# (make sure  ES_HEAP_SIZE is large enough) 
# Parsing whole document xml.cElementTree.parse()
# Streaming the xml document: xml.sax.reader.html

# import xml.etree.ElementTree as etree
# for event, elem in etree.iterparse(xmL, events=('start', 'end', 'start-ns', 'end-ns')):
#  print event, elem
# http://boscoh.com/programming/reading-xml-serially.html
# Event handlers 

## Query system 

* Normalise query
* Get right tokens from the query. Use patterns to split up the query in parts? 
* Put them in the right representation for ES search

In [63]:
def search(query, advanced=False):
    '''
    Given a query it returns a SERP with rakings based on the score
    '''
    if advanced:
        must = []
        if query[0]:
            for i in query[0]:
                must.append({"term": {"_type": i}})
    
        q = {"query": 
                {"filtered": 
                    {"query": {
                        "multi_match": 
                            {"query" : query[1],
                             "type" : "cross_fields",  # with 'and' operator this is strict
                             "fields" : query[2],
                             "operator" : 'and'
                            }
                        },
                     "filter": 
                        [{"bool" : 
                            {"should" : must
                            }
                        },{'range':
                            {"year":
                             {"gte": query[3][0], 
                              "lte": query[3][1]}
                                
                            }}]
                    }
                }
            }
        res = es.search(index='telegraaf', size=10, body=q)
        return res
    else:
        # filter_path can help reduce the amount of data that is returned by the es.search
        # The query context is for how well the document fits the query
        # The filter context is a boolean context. Does it match or not.
        # example: Does this timestamp fall into the range 2015 to 2016?
        #
        
        # The outer 'query': is necessary to show that this is the query.  
        q = {'query':
                {'multi_match':
                    {'query' : query,
                     'type' : 'cross_fields',  # with 'and' operator 
                     'fields' : ['title', 'text'],
                     'operator' : 'and'
                     }
                 }
             }
        # In other words, all terms must be present in at least one field for a document to match.
        res = es.search(index='telegraaf', size=10, body=q)
        return res
    

In [5]:
res = search('stoomschip engelsche')

In [6]:
res['hits']['total']

4

## Result page function

* Take query output and use score to order result on a Search Engine Result Page (SERP).
* Return title, link, and description of each hit

-> The description can be a word cloud of 20-25 most informative words. Represent the hits of a query with a wordcloud of 25-50 informative words. The wordcloud should somehow summarise what the collection has to say about the query. You may think of these words as words that you could add to the query in order to improve recall (blind relevance feedback/query expansion). 


Additions
* A timeline with the amount of hits over time
* A table with the number of hits for each political party. Link the party names, which should result in only selecting the hits "ingediend" by members of that party. (Faceted Search) (For the "Telegraaf" collectie, use the dc:subject element as facet values.)

## Advanced Search

The query system will have to be changed to implement this

* Make multiple fields searchable:
    * Title 
    * Tekst
    * Year?
    
Let a user be able to search in several fields, also in several fields simulteanously. Queries like "return kamervragen by Wilders about XXX with an answer about YYY in the period ZZZ" should be possible. (For the "Telegraaf" collectie, let the user search in both title and tekst fields)

In [7]:
# unique_years = list(set( document.date.get_text()[:4]
#                     for document in soup_documents ))

In [58]:
# Determine values for the year facets
agg={
    "aggs" : {
        "_source" : {
            "terms" : { "field" : "year", "size" : len(documents) }
            
        }
    }
}

agg2={
    "aggs" : {
        "_type" : {
            "terms" : { "field" : "_type" }
            
        }
    }
}

# Get field values for the year
res = es.search(index='telegraaf', body=agg)
unique_years_string = sorted([ "%s (%d documents)" % (item['key'], item['doc_count']) 
                      for item in res['aggregations']['_source']['buckets']])
unique_years = sorted([item['key'] for item in res['aggregations']['_source']['buckets']])

# Get field values for document type
res = es.search(index='telegraaf', body=agg2)
unique_doc_types_string = [ "%s (%d documents)" % (item['key'], item['doc_count']) 
                            for item in res['aggregations']['_type']['buckets']]
unique_doc_types = [item['key'] for item in res['aggregations']['_type']['buckets']]

print unique_years


# # Determine values for the document type facets
# doc_types = list(es.indices.get_mapping(index='telegraaf')['telegraaf']['mappings'])
# unique_doc_types = set(doc_types)

# a = es.indices.get_mapping(index='telegraaf')

[u'1918', u'1922', u'1923', u'1951', u'1961', u'1962', u'1963', u'1965', u'1966', u'1968', u'1969', u'1970', u'1971', u'1972', u'1973', u'1974', u'1975', u'1976', u'1977', u'1978', u'1979', u'1980', u'1981', u'1982', u'1983', u'1984', u'1985', u'1986', u'1987', u'1988', u'1989', u'1990', u'1991', u'1992', u'1993', u'1994']


In [261]:
import formlayout 

def query_database():
    query = formlayout.fedit([('Document type',[0]+unique_doc_types_string),
                   ('Zoektermen',''),
                   ('Jaar publicatie',[0]+unique_years_string)],
                   title="Telegraaf zoekmachine", 
                   comment="Wat voor krantenartikel zoek je?")

    query[0] = unique_doc_types[query[0]]

    res = search(query, advanced=True)
    serp = result_page(query[1], res['hits']['hits'], "http://kranten.kb.nl/view/article/id/", 50, 15)
    results = formlayout.fedit(serp, title="Telegraaf results")

In [8]:
import math

def position_sentences(positions, text, m):
    """
    Return a sentence in which multiple m words 
    from the text occur, based on a list of positions.
    """
    mini = positions[0]
    maxi = mini
    for i in positions[1:]:
        if i > mini and i <= mini + m:
            maxi = i
    diff = int(math.floor((m - (maxi - mini)) / 2))
    return '...'+' '.join(text[mini-diff:maxi+diff])+'...'


def extract_description(query, text, m):
    """
    Given a query, select m words from the text that
    contain words from the query.
    """
    query = query.split()
    stext = text.split(' ')
    positions = []
    # get the word position 
    for word in query: 
        for i,term in enumerate(stext):
            if word in term:
                positions.append(i)
    positions = [i for i in sorted(positions) if i > 7]

    # If word(s) appeared in text, return these sentences
    if positions:
        description = position_sentences(positions, stext, m)
    # If the word only occured in title, return first sentence/part of first sentence
    else:
        description = ''.join(text.split('. ')[0]) + '...'
    return description


def result_page(query,total_hits, hits):
    total = widgets.HTML('Total hits: '+str(total_hits)+" Shown: 10")
    results = []
    descriptions = []
    for elem in hits:
        if elem['_source']['title'] == '':
            results.append(widgets.HTML('<h3><a href="http://kranten.kb.nl/view/article/id/'+ str(elem['_id'])+'" target="_blank">No Title Available</a></h3>'))
        else:
            results.append(widgets.HTML(value = '<h3><a href="http://kranten.kb.nl/view/article/id/'+ str(elem['_id'])+'" target="_blank">'+elem['_source']['title']+'</a></h3>'))
        results.append(widgets.HTML(extract_description(query, elem['_source']['text'],15)))
    return results

# titles, description = result_page('car', res['hits']['total'], res['hits']['hits'])
# for t, d in zip(titles,description):
#     display(t)
#     display(d)
    

In [65]:
from IPython.display import display, clear_output
from IPython.core.display import HTML
from ipywidgets import widgets, interact
# res = search('stoomschip')
# print res['hits']['hits'][0]['_source']

def get_count(ty, query, fields):
    return es.count(index='telegraaf',body={'query':{"filtered":{"query":
                                                                     {"multi_match": 
                                                                        {"query" : query,
                                                                         "type" : "cross_fields",  # with 'and' operator this is strict
                                                                         "fields" : fields,
                                                                         "operator" : 'and'
                                                                        }
                                                                    },
                                                                    "filter":
                                                                    {"bool":
                                                                        {"must": {"term": {"_type": ty}}}
                                                                     }
                                                                    }
                                                        }
                                               })['count']



SERP = ''

# Create de search field and the search button
text = widgets.Text(placeholder='Vul een zoekterm in')
search_button = widgets.Button(description="Search")
advanced = widgets.ToggleButton(description="Toggle Advanced Search", width='220px', button_style='success', value=True)
container = widgets.HBox((widgets.HTML('',width='130px'),
                          widgets.HTML('Zoektermen:'),
                          text, 
                          widgets.HTML(''), 
                          search_button,
                          widgets.HTML('',width='50px'),
                          advanced))
container.layout.align_items='center'
HBox_layout = widgets.Layout(align_items='center'
                            ,align_content='center')

#Text field checkbox
text_check = widgets.Checkbox(value=True)
textfield = widgets.HBox(children=(widgets.HTML('',width='20px'),
                                   widgets.HTML('Text'),
                                   text_check))
textfield.layout.align_items='center'

#Title field checkbox
title_check = widgets.Checkbox(value=True)
titlefield = widgets.HBox(children=(widgets.HTML('',width='20px'),
                                    widgets.HTML('Title'),
                                    title_check))
titlefield.layout.align_items='center'
# Container for searchfields
c_textfields = widgets.VBox(children=(widgets.HTML('Welke zoekvelden?'),
                                      textfield, 
                                      titlefield))
titlefield.layout.align_items='center'


# The type fields: 
class doc_widget:
    
    def __init__(self,name,value):
        self.value = value
        self.name = name
        self.widget = widgets.Text(str(self.name)+' ('+str(self.value)+')', width='220px', disabled=True)
        
    def set_value(self, value):
        self.value = value
        self.widget = widgets.Text(str(self.name)+' ('+str(self.value)+')', width='220px', disabled=True)
    
doc_types = {}
for i in unique_doc_types:
    doc_types[i]  = doc_widget(i,0)

ad_check = widgets.Checkbox(value=True, width='20px')
ar_check = widgets.Checkbox(value=True, width='20px')
io_check = widgets.Checkbox(value=True, width='20px')
fb_check = widgets.Checkbox(value=True, width='20px')

row1 = widgets.HBox(children=(ad_check,
                              doc_types['advertentie'].widget, 
                              ar_check,
                              doc_types['artikel'].widget))
row1.layout.align_items='center'

row2 = widgets.HBox(children=(io_check,
                              doc_types['illustratie met onderschrift'].widget,
                              fb_check,
                              doc_types['familiebericht'].widget))
row2.layout.align_items='center'
c_types = widgets.VBox(children=(widgets.HTML('Welke type documenten?'),
                                 row1,
                                 row2))

# The time period range:
years = widgets.IntRangeSlider(value=[int(unique_years[0]), int(unique_years[-1])], 
                               min=int(unique_years[0]),
                               max= int(unique_years[-1]), 
                               step=1)

c_slide = widgets.VBox(children=(widgets.HTML("Kies tijdsperiode:"), 
                                 years))


# Advanced settings container
container_adv = widgets.HBox((c_textfields,   
                              c_types,     # misschien een extra beetje ruimte tussen plaatsen   
                              c_slide))    # Slider to select the number of years
container_adv.visible=True


def advanced_options(sender):
    if sender['new'] == True:
        container_adv.visible = True
        # Show the advanced settings
        
    elif sender['new'] == False:
        container_adv.visible = False
        # Hide the advanced settings

        
    
        
def handle_submit(sender):
    '''
    This function handles the search after the button has been presed or enter in the text field
    '''
    global SERP 
    if SERP != '':
        SERP.close()
        
    # The simple search function were it just searches for the keywords
    if container_adv.visible == False:
        display('Zoeken..')
        res = search(text.value)
        clear_output()
        results = result_page(text.value, res['hits']['total'], res['hits']['hits'])
        if results == []:
            SERP = widgets.VBox(children=(widgets.HTML('', height='20px'),
                                          widgets.HTML("<center>Geen zoekresultaten zijn gevonden. Probeer het opnieuw</center>")))
        else:
            SERP = widgets.VBox(children=(tuple([i for i in results])))
        display(SERP)
        
    # The advanced search
    else:
        display('Zoeken..')
        
        # Get which types are checked for the search
        types = []
        if ad_check.value:
            types.append('advertentie')
        if ar_check.value:
            types.append('artikel')
        if io_check.value:
            types.append('illustratie met onderschrift')
        if fb_check.value:
            types.append('familiebericht')
        
        fields = []
        # Which text fields? 
        if title_check:
            fields.append('title')
        if text_check:
            fields.append('text')
        
        # Get values from slider
        
        
        # Search 
        res = search([types,text.value,fields,years.value], advanced=True)
        clear_output()
        results = result_page(text.value, res['hits']['total'], res['hits']['hits'])
        
        # Print the outputs
        if results == []:
            SERP = widgets.VBox(children=(widgets.HTML('', height='20px'),
                                          widgets.HTML("<center>Geen zoekresultaten zijn gevonden. Probeer het opnieuw</center>")))
        else:
            SERP = widgets.VBox(children=(tuple([i for i in results])))
        display(SERP)
        
        # update the type doc numbers
        for i in doc_types:
            doc_types[i].widget.value= doc_types[i].name+' ('+str(get_count(i,text.value,fields))+')'
        
        
        

        
    
    
display(container)
display(container_adv)
text.on_submit(handle_submit)
search_button.on_click(handle_submit)
advanced.observe(advanced_options,names='value')
# advanced.on_click(handle_submit)
# help(advanced)
# Submit the search 



# Show the advanced search options
# def advanced_options():
    


## Result page function

* Take query output and use score to order result on a Search Engine Result Page (SERP).
* Return title, link, and description of each hit

### SERP & Worcloud
Since stopwords have high frequencies, they are likely to occupy most places in the word cloud. We therefore remove an extensive stopword list consisting of 571 common English words. Only single words (unigrams) are included in the cloud and stemming is applied. To create a word cloud all terms in the document are sorted by their probabilities and a fixed number of the 25 top ranked terms are kept. The top 10 documents retrieved by a language model run are concatenated and treated as one long document.

In [34]:
from stop_words import get_stop_words
import snowballstemmer
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from collections import Counter
import math
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

def result_page(query, hits, url, n, m):
    """
    Return the SERP information from a given list of
    search results.
    """
    if hits:
        serp = []
        total_text = []
        years = []
        dates = []
        for i, hit in enumerate(hits[:10]):
            year = hit['_source']['year']
            years.append(year)
            dates.append(hit['_source']['date'])
            identifier = url+str(hit['_id'])
            serp.append((None,None))
            serp.append((None,"Jaar publicatie: %s, Onderwerp: %s" % (year,hit['_type'])))
            serp.append((None, "URL: <a href='%s'>%s</a>" % (identifier,identifier)))
            serp.append((None,hit['_source']['title']))
            text = hit['_source']['text'].split()
            total_text = total_text + text
            serp.append((None,"Beschrijving: " + extract_description(query,text,m)))
        wordcloud = create_wordcloud(text, n)
        timeline = create_timeline(years,dates)
    else:
        serp = [(None,'\n\n\n\nEr zijn helaas geen zoekresultaten.\n\n\n\n')]
    return serp

def create_wordcloud(text, n):
    """
    Display a wordcloud with at most n words, generated
    from the given text.
    """
    # Filter words to use for the wordcloud, by stemming and stop words removal
    stop_words = get_stop_words("dutch")
    stemmer = snowballstemmer.stemmer("dutch")
    text = [word for word in text if word.lower() not in stop_words]
    text = stemmer.stemWords(text)

    # Plot wordcloud
    wordcloud = WordCloud(max_font_size=40, background_color="white",
                          max_words = n).generate(" ".join(text))
    plt.figure()
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.show()

def extract_description(query, text, m):
    """
    Given a query, select m words from the text that
    contain words from the query.
    """
    query = query.split()
    positions = sorted([text.index(word) for word in query 
                        if word in text ])
    
    positions = [i for i in positions if i > 7]

    # If word(s) appeared in text, return these sentences
    if positions:
        description = position_sentences(positions, text, m)
    # If the word only occured in title, return first sentence
    else:
        description = ' '.join(text[0:m]) + '...'
    
    return description

def position_sentences(positions, text, m):
    """
    Return a sentence in which multiple m words 
    from the text occur, based on a list of positions.
    """
    mini = positions[0]
    maxi = mini
    for i in positions[1:]:
        if i > mini and i <= mini + m:
            maxi = i
    diff = int(math.floor((m - (maxi - mini)) / 2))
    return '...'+' '.join(text[mini-diff:maxi+diff])+'...'

def create_timeline(years, dates):
    """
    Display the years or dates from the hits
    on a timeline.
    """

    ax = plt.figure(figsize=(15,4)).gca()
    ax.yaxis.set_major_locator(MaxNLocator(integer=True))
    
    if len(set(years)) == 1:
        xlabel = "Months of the year" + years[0]
        bins = [ int(date[:2]) for date in dates ]
        #counted = Counter(months)
        names = ['           Januari','             Februari','         Maart',
                 '        April', '        Mei','        Juni','        Juli',
                 '               Augustus','                 September',
                 '               Oktober','                November',
                 '                December']
        xlim = [1,13]
        plt.xticks(range(1,13),names)
        bin_range = range(1,13)
    else: 
        bins = years 
        bin_range = range(min(years),max(years)+1)
        xlabel = "Years"
        xlim = [min(years),max(years)]

    ax.set_xlim(xlim)
    
    # Mooi roze is niet leelijk
    plt.hist(bins, bins=bin_range, color='Crimson')
    plt.xlabel(xlabel)
    plt.ylabel("Number of documents")
    plt.show()
    
#create_timeline([1922,1992],['11-12','10-31','11-6'])        

ImportError: No module named stop_words

## Evaluation

* Manual relevance check
* P@10
* Change the ranking of the system + explain what is going on and why it is improving/decreasing

Evaluate your results Let 2 persons assess the relevancy of the top 10 documents for 5 different queries. Compute Cohen's kappa. Determine the average precision at 10 for your system based on these 10 queries, and the two relevance assesments. Also plot the P@10 (for both judges) for each query, showing differences in hard and easy queries. Describe clearly how you solved differences in judgements. 
So, both provide the actual query, and a description of the information need that was behind the query.
Give a small set of clear guidelines for judging the results, and let your judges follow these guidelines.
It is far more interesting to have difficult queries (both for the search engine and for the judges) than to have queries on which all ten retrieved documents are relevant. So, try to create a good list of information needs.


In [51]:
query_database()

[u'familiebericht', u'overlijden', u'1981']


In [6]:
from __future__ import division

def cohens_kappa(bins):
    '''
    Given bins made by bin_evaluations, returns cohen\'s kappa
    '''
    
    total = bins[0]+bins[1]+bins[2]+bins[3]
    po = (bins[0]+bins[3]) / total
    
    marginala = ((bins[3] + bins[2]) * (bins[3] + bins[1])) / total
    marginalb = ((bins[0] + bins[1]) * (bins[0] + bins[2])) / total
    pe = (marginala + marginalb) / total
    
    cohens_kappa = (po - pe) / (1 - pe)
    return cohens_kappa
    
def precision(bins, agreement_necessary):
    '''
    Given bins made by bin_evaluations, returns precision
    If second argument is True, document is only seen as
    relevant if both judges think so; otherwise one of the 
    judges is enough
    '''
    if agreement_necessary:
        correct = bins[3]
    else:
        correct = bins[3] + bins[2] + bins[1]
    total = bins[0]+bins[1]+bins[2]+bins[3]
    return correct/total
    
    
def bin_evaluations(evaluations):
    '''
    Given a list of tuples of binary values which contain
    relevance judgements by two judges in which 1 means 
    relevant, and 0 means non-relevant, returns a list
    in which the 4 elements represent the amount of times
    any combination has occurred.
    '''
    
    
    # bins are 00, 01, 10, 11 in that order
    bins = [0,0,0,0]
    for evaluation in evaluations:
        b = 0
        if evaluation[0]:
            b += 2
        if evaluation[1]:
            b += 1
        bins[b] += 1
    return bins

In [44]:
def evaluate_search_results(number_of_queries):
    '''
    Asks the user to submit a query using query_database() some amount 
    of times after which two judges can give their relevancy 
    assessments. Returns evaluations in a format that can be used for 
    bin_evaluations.
    '''

    print('After the query is resolved, you will be asked to rate the documents on relevancy.')
    print('To do this, enter the numbers of the relevant results sperated by whitespace.')
    print('For example, if only the first and third documents were relevant, enter \'1 3\' without the quotes.')
    print('If more than ten documents are returned, ignore all but the first 10.\n')
    evaluations = []
    for _ in range(number_of_queries):
        query_database()
        print('Indicate which documents were relevant:')
        judge1_input = raw_input('Judge 1 -->').split()
        judge2_input = raw_input('Judge 2 -->').split()
        for i in range(1, 11):
            if str(i) in judge1_input:
                assessment1 = 1
            else:
                assessment1 = 0
            if str(i) in judge2_input:
                assessment2 = 1
            else:
                assessment2 = 0
            evaluations.append((assessment1, assessment2))
    
    bins = bin_evaluations(evaluations)
    print('\n The average P@10 if we require agreement for correctness: ' + str(precision(bins, 1)))
    print("The average P@10 if we require a single \'relevant\' assessment for correctness: " + str(precision(bins, 0)))
    print("Cohen\'s Kappa was: " +str(cohens_kappa(bins)))
    return evaluations

In [47]:
evaluations = evaluate_search_results(5)

After the query is resolved, you will be asked to rate the documents on relevancy.
To do this, enter the numbers of the relevant results sperated by whitespace.
For example, if only the first and third documents were relevant, enter '1 3' without the quotes.
If more than ten documents are returned, ignore all but the first 10.

Indicate which documents were relevant:
Judge 1 -->1
Judge 2 -->1
Indicate which documents were relevant:
Judge 1 -->2
Judge 2 -->2

 The average P@10 if we require agreement for correctness: 0.1
The average P@10 if we require a single 'relevant' assessment for correctness: 0.1
Cohen's Kappa was: 1.0
